In [52]:
import tensorflow as tf
import pandas as pd
import tempfile
sess = tf.InteractiveSession()

In [53]:
data = pd.DataFrame({
    'x': range(10),
    'y': range(10)
})
file_path = 'data_weight.csv'
data.to_csv(file_path, index=False, header=False)
data

,x,y
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9


In [55]:
_CSV_COLUMN_DEFAULTS = [[0],[0]]
_CSV_COLUMNS = ['x', 'y']

def input_fn(data_file):

    def parse_csv(value):
        print('Parsing', data_file)
        columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
        features = dict(zip(_CSV_COLUMNS, columns))
        labels = features.pop('y')
        return features, labels

    # Extract lines from input files using the Dataset API.
    dataset = tf.data.TextLineDataset(data_file)
    dataset = dataset.map(parse_csv)

    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

In [57]:
x = tf.feature_column.numeric_column('x')
base_columns = [x]

model_dir = tempfile.mkdtemp()
model = tf.estimator.LinearRegressor(model_dir=model_dir, feature_columns=base_columns)

model = model.train(input_fn=lambda: input_fn(data_file=file_path))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfmk0fmz0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94fd8f4320>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Parsing data_weight.csv


ValueError: Feature (key: x) cannot have rank 0. Give: Tensor("IteratorGetNext:0", shape=(), dtype=int32, device=/device:CPU:0)